### Mini Project | Real Time Data Analytics | Sentiment Analysis on Twitter in Real Time
#### Group Members
* P Kumaralingam
* Geetha Viswanathan
* Madhavan
* Paraneetharan M

This mini project is to perform a Sentiment Analysis on Twitter data using Spark Streaming in Real Time. There are 3 parts in this project which contains code to receive twitter data and perform several analysis in real time. In this project we have used Kafka as Messaging Agent (similar to socket). Here Spark Streaming is consuming the twitter data in every 10 secs.

## PART 1 - Create a Streaming Listener (using Kafka) and Push Twitter Data into it

In [14]:
# Twitter libraries
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

# Kafka libraries
from kafka import KafkaProducer
from kafka import KafkaClient

# Other libraries
import sys, time, json 

In [15]:
# Twitter Consumer & Access Keys
consumer_key = 'hLLsDTJ6Wl6IbUZIC2hkLq9Gv'
consumer_secret = '5wDW5Jy28ZHfM4fS3zndwA3Il6kc3vnW9O5XGTC6e0rocLn0Ty'
access_token = '173511847-S0jjSl51SJsiv55UIdPnPtVHdxRotzNWAXtVDg3z'
access_secret = 'idBhq9YBx0reNYv8KM2csEqegzm53tiGXLGIKr0ZPAeRg'

In [16]:
# Set Consumer & Access Keys
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [17]:
# Create KafkaClient
client = KafkaClient(hosts="ip-20-0-32-4.ap-south-1.compute.internal:9092")

In [18]:
# Create Kafka Topic if not already exist
topic = 'rtd2analysis'
client.ensure_topic_exists(topic)

#### Setup Kafka Producer to push the tweets into the topic
Note: Here we use the broker server from Cloud Lab.

In [19]:
producer = KafkaProducer(bootstrap_servers=['ip-20-0-32-4.ap-south-1.compute.internal:9092'],
                         key_serializer=lambda m: m.encode('utf8'),
                         value_serializer=lambda m: json.dumps(m).encode('utf8'))

Variable tweets_limit is used to control the number of tweets streamed from Twitter API. It helps to prevent the Kafka Producer to run for longer duration by stop the same automatically after the limit is reached.

In [20]:
# Streaming Listener to push tweets using Kafka Producer created in above step
class KafkaListener(StreamListener):
    def __init__(self, tweets_limit):
        self.counter = 0
        self.tweets_limit = tweets_limit
        
    def on_data(self, data):
        tweet = json.loads(data)
        
        # Filter and send only tweet data (skip delete data)
        if "created_at" in tweet:
            self.counter += 1
            producer.send(topic,key=tweet["created_at"],value=tweet)
            if (self.counter % 1000 == 0):
                print ("Limit reacehd to ",self.counter)
            
        if self.counter == self.tweets_limit:
            print ("Max Tweet Limit Reached..!")
            return False

    def on_error(self, status):
        print(status)
        print("Rate limited..")
        return False

In [22]:
#This handles Twitter authetification and the connection to Twitter Streaming API
if __name__ == '__main__':
    
    try:
        l = KafkaListener(tweets_limit=20000)
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        stream = Stream(auth, l)
        # Sample delivers a stream of 1% (random selection) of all tweets
        stream.sample()
        
        #Set Keywords to filter Twitter Streams. It will be used for iphone Sentiment Analysis
        #stream.filter(languages=['en'],track=['iphone'])  

    except KeyboardInterrupt:
        sys.exit(0)

('Limit reacehd to ', 1000)
('Limit reacehd to ', 2000)
('Limit reacehd to ', 3000)
('Limit reacehd to ', 4000)
('Limit reacehd to ', 5000)
('Limit reacehd to ', 6000)
('Limit reacehd to ', 7000)
('Limit reacehd to ', 8000)
('Limit reacehd to ', 9000)
('Limit reacehd to ', 10000)
('Limit reacehd to ', 11000)
('Limit reacehd to ', 12000)
('Limit reacehd to ', 13000)
('Limit reacehd to ', 14000)
('Limit reacehd to ', 15000)
('Limit reacehd to ', 16000)
('Limit reacehd to ', 17000)
('Limit reacehd to ', 18000)
('Limit reacehd to ', 19000)
('Limit reacehd to ', 20000)
Max Tweet Limit Reached..!
